In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import shapely
from shapely import wkt
import h3
from datetime import date
import pyarrow.parquet as pq
import pyarrow as pa
#from fastparquet import write
#from parquet

# Data Cleaning

In [2]:
df = pd.read_parquet("../data/df_sample_27_05_2021.parquet")
df["PU_Centroid"] = gpd.points_from_xy(df["Pickup Centroid Latitude"], df["Pickup Centroid Latitude"])
df["DO_Centroid"] = gpd.points_from_xy(df["Dropoff Centroid Latitude"], df["Dropoff Centroid Latitude"])                                                   
#df = df.drop(columns = ["Pickup Centroid Latitude","Pickup Centroid Longitude","Dropoff Centroid Latitude","Dropoff Centroid Longitude"])
df["Trip Start Timestamp"] = pd.to_datetime(df["Trip Start Timestamp"],format = '%m/%d/%Y %I:%M:%S %p')
df["Trip End Timestamp"] = pd.to_datetime(df["Trip End Timestamp"],format = '%m/%d/%Y %I:%M:%S %p')
df.head()

,Trip ID,Taxi ID,Trip Start Timestamp,Trip End Timestamp,Trip Seconds,Trip Miles,Pickup Census Tract,Dropoff Census Tract,Pickup Community Area,Dropoff Community Area,...,Payment Type,Company,Pickup Centroid Latitude,Pickup Centroid Longitude,Pickup Centroid Location,Dropoff Centroid Latitude,Dropoff Centroid Longitude,Dropoff Centroid Location,PU_Centroid,DO_Centroid
11871910,aeb44747d9cd5c4e810ed2c12631362d827ae29a,bb4ff740f6b2ffbe9aa3900f462c265eae88dadab4a095...,2017-06-14 13:15:00,2017-06-14 13:30:00,300.0,0.8,1.703108e+10,1.703108e+10,8.0,8.0,...,Cash,Top Cab Affiliation,41.892042,-87.631864,POINT (-87.6318639497 41.8920421365),41.899156,-87.626211,POINT (-87.6262105324 41.8991556134),POINT (41.89204 41.89204),POINT (41.89916 41.89916)
4278691,f3150e8061916e2cec7dad53f038f9d7cac01101,8e856e16163a85cee43d1ca81190e3b369b5a3724fe1c1...,2017-03-06 14:45:00,2017-03-06 15:00:00,1020.0,9.5,NaN,NaN,NaN,NaN,...,Cash,Chicago Independents,NaN,NaN,None,NaN,NaN,None,POINT (nan nan),POINT (nan nan)
962707,3bc4f08fcf057a6c2bd8b7865c1978abca3616ec,1721f8d69b5f831e0c16c51510786ca43aaa367d23bec8...,2017-01-16 11:45:00,2017-01-16 12:00:00,392.0,2.2,NaN,NaN,NaN,NaN,...,Credit Card,303 Taxi,NaN,NaN,None,NaN,NaN,None,POINT (nan nan),POINT (nan nan)
21548285,7235674388b12e378a2bc20ad7fb565ef6a9d317,61abb47c5869d156b0658c031a16ce139beb682fc30b5b...,2017-11-04 00:00:00,2017-11-04 00:15:00,1260.0,4.7,NaN,NaN,7.0,24.0,...,Cash,City Service,41.922686,-87.649489,POINT (-87.6494887289 41.9226862843),41.901207,-87.676356,POINT (-87.6763559892 41.90120699410001),POINT (41.92269 41.92269),POINT (41.90121 41.90121)
16198848,4ab70436ec0c42ae288afb510213264fd3c6e5c3,ea73ddfdd7cafa796ab1e254ffe3240fb6936e6f390c27...,2017-08-16 12:45:00,2017-08-16 13:00:00,423.0,1.1,1.703108e+10,1.703132e+10,8.0,32.0,...,Credit Card,Blue Diamond,41.895033,-87.619711,POINT (-87.6197106717 41.8950334495),41.884987,-87.620993,POINT (-87.6209929134 41.8849871918),POINT (41.89503 41.89503),POINT (41.88499 41.88499)


In [3]:
df_filtered = df.nsmallest(int(df.index.size * 0.999), "Trip Total", keep='first')
total_filter = df_filtered["Trip Total"].max()
min_total = df_filtered["Trip Total"].min()
print("Max Total:",total_filter)
print("Min Total:",min_total)

df_filtered = df.nsmallest(int(df.index.size * 0.999), "Trip Seconds", keep='first')
seconds_filter = df_filtered["Trip Seconds"].max()
min_seconds = df_filtered["Trip Seconds"].min()
print("Max Seconds:",seconds_filter)
print("Min Seconds:",min_seconds)

df = df.copy()[(df["Trip Total"] <= total_filter)&
                   (df["Trip Seconds"]<=seconds_filter)&
                   (df["Trip Seconds"]> 60)]
print("Kept",np.round(df.index.size / df.index.size,4),"percent of data")

Max Total: 124.5
Min Total: 0.0
Max Seconds: 7063.0
Min Seconds: 0.0
Kept 1.0 percent of data


In [4]:
#kick out values under 2$
print("0$ Total Trips:",df[df["Trip Total"]==0].index.size)
print("0$ Fare Trips:",df[df["Fare"]==0].index.size)


df = df.copy()[(df["Trip Total"] >= 2)&
                   (df["Fare"]>=2)]

print("New min Total: ",min(df["Trip Total"]))
print("New max Total: ",max(df["Trip Total"]))
print("New min Fare: ",min(df["Fare"]))
print("New max Fare: ",max(df["Fare"]))

print("Kept",np.round(df.index.size / df.index.size,4),"percent of data")

0$ Total Trips: 350
0$ Fare Trips: 414
New min Total:  2.0
New max Total:  124.5
New min Fare:  2.0
New max Fare:  124.5
Kept 1.0 percent of data


In [5]:
print("New min Total: ",min(df["Trip Total"]))
print("New max Total: ",max(df["Trip Total"]))

New min Total:  2.0
New max Total:  124.5


In [6]:
#For understanding 
df[["Trip Total","Tolls","Tips","Extras","Fare"]].sample(10)

,Trip Total,Tolls,Tips,Extras,Fare
16398790,19.00,0.0,0.00,0.0,19.00
10523715,15.90,0.0,2.65,1.0,12.25
2688215,16.25,0.0,0.00,0.0,16.25
10130077,49.35,0.0,9.85,4.0,35.50
6899475,52.80,0.0,8.80,4.0,39.50
19256994,8.25,0.0,0.00,0.0,8.25
16322881,6.00,NaN,0.00,1.0,5.00
10486347,8.75,NaN,0.00,0.0,8.75
13375998,5.00,0.0,0.00,0.0,5.00
16784937,40.25,0.0,5.00,4.0,31.25


In [7]:
#Kick out all values of December and 2018
df['start_month'] = df['Trip Start Timestamp'].dt.month
df['start_year'] = df['Trip Start Timestamp'].dt.year
df['end_month'] = df['Trip End Timestamp'].dt.month
df['end_year'] = df['Trip End Timestamp'].dt.year

df = df[(df['start_year'] == 2017 ) & (df['end_year'] == 2017) ]
df = df[(df['start_month'] !=12 ) & (df['end_month'] !=12) ]
print("Kept",np.round(df.index.size / df.index.size,4),"percent of data")

#Drop columns again
df.drop(columns=['start_month', 'start_year', 'end_month' , 'end_year'], inplace = True)

print()
#Min and Max Trip Start and end
print("New min start: ",min(df['Trip Start Timestamp']))
print("New min end: ",min(df['Trip End Timestamp']))
print("New min start: ",max(df['Trip Start Timestamp']))
print("New min end: ",max(df['Trip End Timestamp']))

Kept 1.0 percent of data

New min start:  2017-01-01 00:00:00
New min end:  2017-01-01 00:00:00
New min start:  2017-11-30 23:45:00
New min end:  2017-11-30 23:45:00


In [8]:
#Check null values in payment type and company
pay_verifier = df['Payment Type'].dropna()
print("Number of null values within column payment type: ",len(df)-len(pay_verifier))

com_verifier = df['Company'].dropna()
print("Number of null values within column payment type: ",len(df)-len(com_verifier))
print()
print("if 0, we don't have to drop something.")

Number of null values within column payment type:  0
Number of null values within column payment type:  0

if 0, we don't have to drop something.


In [24]:
#Miles have to be checked with the geo data

# THIS HAS TO BE CHECKED!! - parquet actually without this steps!

In [14]:
#Converting Geo-Points to H3

def h3_conversion(value,h3_level):
    if isinstance(value,shapely.geometry.point.Point):
        return h3.geo_to_h3(value.y, value.x, h3_level)
    else:
        return np.nan

df["PU_H3"] = df.apply(lambda x: h3_conversion(x["PU_Centroid"],7),axis=1)
df["DO_H3"] = df.apply(lambda x: h3_conversion(x["DO_Centroid"],7),axis=1)
df

,Trip ID,Taxi ID,Trip Start Timestamp,Trip End Timestamp,Trip Seconds,Trip Miles,Pickup Census Tract,Dropoff Census Tract,Pickup Community Area,Dropoff Community Area,...,Extras,Trip Total,Payment Type,Company,Pickup Centroid Location,Dropoff Centroid Location,PU_Centroid,DO_Centroid,PU_H3,DO_H3
5402408,55e9f8e9d439cbe136ba82622ede4be417b4e760,c1365fc5d9c26f99b42911754818c3402e38e0094c19e4...,2017-03-20 17:45:00,2017-03-20 18:00:00,666.0,13.40,NaN,NaN,76.0,NaN,...,4.0,37.50,Cash,Flash Cab,POINT (-87.913624596 41.9802643146),None,POINT (41.98026 41.98026),POINT (nan nan),872c21010ffffff,0
18781727,f2a95893fef0e4f7141d0927f219d65839ec3f97,e203f043bc67df3a0c247621faa4524d93d89a49b02457...,2017-09-25 09:00:00,2017-09-25 09:00:00,714.0,1.11,1.703128e+10,1.703132e+10,28.0,32.0,...,0.0,8.00,Cash,Chicago Carriage Cab Corp,POINT (-87.6428084655 41.8853000224),POINT (-87.6209929134 41.8849871918),POINT (41.88530 41.88530),POINT (41.88499 41.88499),872c2119cffffff,872c2119cffffff
19602125,9a3c7914c71fdb5fc3bf1794f171b712aa81b901,7b885f3c3b55b58bb3785c7b6876f9d5bf3d8a3e74486b...,2017-10-07 04:15:00,2017-10-07 04:30:00,480.0,0.00,NaN,NaN,28.0,7.0,...,0.0,12.00,Cash,Taxi Affiliation Services,POINT (-87.6635175498 41.874005383),POINT (-87.6494887289 41.9226862843),POINT (41.87401 41.87401),POINT (41.92269 41.92269),872c21191ffffff,872c2118affffff
215156,509af19342d154339c072d227c3bb648c3ad3ab2,0bd46f4637ddcdab1350b60cb03cbd9281279a7bb4a049...,2017-01-05 08:00:00,2017-01-05 08:15:00,900.0,5.60,1.703106e+10,1.703184e+10,6.0,32.0,...,0.0,17.00,Cash,City Service,POINT (-87.640698076 41.9431550855),POINT (-87.6327464887 41.8809944707),POINT (41.94316 41.94316),POINT (41.88099 41.88099),872c2118bffffff,872c21191ffffff
4668771,edef4a33874a307b48bf18028a47e5ade9645941,300d94fbea035e07d9e812a32051950a7ad66f7de3a3af...,2017-03-11 14:00:00,2017-03-11 14:15:00,1080.0,5.20,1.703108e+10,1.703107e+10,8.0,7.0,...,1.0,21.60,Credit Card,Checker Taxi Affiliation,POINT (-87.6188683546 41.8909220259),POINT (-87.6561568309 41.9289672664),POINT (41.89092 41.89092),POINT (41.92897 41.92897),872c2119cffffff,872c2118affffff
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13113750,cbf2cdc749c6051853f73bcaa0810834b8a3fd9e,93ed0b8d2841e33df894a893a95c6287c2b58779b0529d...,2017-07-01 13:00:00,2017-07-01 13:30:00,2592.0,17.50,1.703108e+10,1.703198e+10,8.0,76.0,...,0.0,50.60,Credit Card,Yellow Cab,POINT (-87.6207628651 41.8983317935),POINT (-87.9030396611 41.9790708201),POINT (41.89833 41.89833),POINT (41.97907 41.97907),872c2119cffffff,872c21010ffffff
15084256,707e4efad78b791a556a0f3cd43c010efe35fae7,185565b2dfcacc95bacae0096afc6dd4bfa39d75a810e9...,2017-07-31 01:45:00,2017-07-31 01:45:00,300.0,1.90,NaN,NaN,24.0,24.0,...,0.0,7.75,Cash,Taxi Affiliation Service Yellow,POINT (-87.6763559892 41.90120699410001),POINT (-87.6763559892 41.90120699410001),POINT (41.90121 41.90121),POINT (41.90121 41.90121),872c2119dffffff,872c2119dffffff
2813457,438017b4550f74dfcb631a20adc7c28ae754261b,fd1428886e44bcaee631154a2aa027fc78c57aea0c405b...,2017-02-13 16:30:00,2017-02-13 16:30:00,240.0,0.60,1.703184e+10,1.703132e+10,32.0,32.0,...,0.0,4.75,Cash,Choice Taxi Association,POINT (-87.6327464887 41.8809944707),POINT (-87.6209929134 41.8849871918),POINT (41.88099 41.88099),POINT (41.88499 41.88499),872c21191ffffff,872c2119cffffff
4766362,00e16f9c42f8d1ede385c227bbb2f2f670df8ea2,8a0cbb4528e81a685396639a29b35d67d6d48bee0f6b96...,2017-03-12 16:45:00,2017-03-12 17:00:00,780.0,0.20,1.703106e+10,1.703107e+10,6.0,7.0,...,0.0,12.75,Cash,Taxi Affiliation Services,POINT (-87.6460070664 41.9534000435),POINT (-87.6559118484 41.9217014922),POINT (41.95340 41.95340),POINT (41.92170 41.92170),872c21016ffffff,872c2118affffff


## Option 2: Fastparquet

# Data Preparation

In [10]:
def addDateCols(added_word, used_datetime,df_name): 

    #added word: Put your indivuword word to the string 

    #Month and weekday name
    df_name[added_word+'_MONTH_NAME'] = df_name[used_datetime].dt.month_name()
    df_name[added_word+'_WEEKDAY_NAME'] = df_name[used_datetime].dt.day_name()
    
    #Year
    df_name[added_word+'_YEAR'] = df_name[used_datetime].dt.year
    df_name[added_word+'_YEAR'] = df_name[added_word+'_YEAR'].astype(str)

    #Month numeric
    df_name[added_word+'_MONTH'] = df_name[used_datetime].dt.month
    df_name[added_word+'_MONTH'] = pd.to_numeric(df_name[added_word+'_MONTH'])
    
    #Weekday numeric
    df_name[added_word+'_WEEKDAY'] = df_name[used_datetime].dt.dayofweek
    df_name[added_word+'_WEEKDAY'] = pd.to_numeric(df_name[added_word+'_WEEKDAY'])

    #Day numeric
    df_name[added_word+'_HOUR'] = df_name[used_datetime].dt.hour
    df_name[added_word+'_HOUR'] = pd.to_numeric(df_name[added_word+'_HOUR'])+1
    
    #Day of week
    df_name[added_word+'_HOUR_OF_WEEK'] = (df_name[added_word+'_WEEKDAY']*24) + df_name[added_word+'_HOUR']
    
    #Fifth of the day
    df_name[added_word+'_FOUR_HOURLY'] = np.where(df_name['PU_HOUR'].isin([1,2,3,4,5]),'1/5', -1)
    df_name[added_word+'_FOUR_HOURLY'] = np.where(df_name['PU_HOUR'].isin([6,7,8,9,10]),'2/5', df_name[added_word+'_FOUR_HOURLY'])
    df_name[added_word+'_FOUR_HOURLY'] = np.where(df_name['PU_HOUR'].isin([11,12,13,14,15]),'3/5', df_name[added_word+'_FOUR_HOURLY'])
    df_name[added_word+'_FOUR_HOURLY'] = np.where(df_name['PU_HOUR'].isin([16,17,18,19,20]),'4/5', df_name[added_word+'_FOUR_HOURLY'])
    df_name[added_word+'_FOUR_HOURLY'] = np.where(df_name['PU_HOUR'].isin([21,22,23,24,25]),'5/5', df_name[added_word+'_FOUR_HOURLY'])
    
    return df_name


In [11]:
#Adding datetime columns
df_prepared = addDateCols("PU","Trip Start Timestamp",df)


In [12]:
df_prepared.head()

,Trip ID,Taxi ID,Trip Start Timestamp,Trip End Timestamp,Trip Seconds,Trip Miles,Pickup Census Tract,Dropoff Census Tract,Pickup Community Area,Dropoff Community Area,...,PU_Centroid,DO_Centroid,PU_MONTH_NAME,PU_WEEKDAY_NAME,PU_YEAR,PU_MONTH,PU_WEEKDAY,PU_HOUR,PU_HOUR_OF_WEEK,PU_FOUR_HOURLY
11871910,aeb44747d9cd5c4e810ed2c12631362d827ae29a,bb4ff740f6b2ffbe9aa3900f462c265eae88dadab4a095...,2017-06-14 13:15:00,2017-06-14 13:30:00,300.0,0.8,1.703108e+10,1.703108e+10,8.0,8.0,...,POINT (41.89204 41.89204),POINT (41.89916 41.89916),June,Wednesday,2017,6,2,14,62,3/5
4278691,f3150e8061916e2cec7dad53f038f9d7cac01101,8e856e16163a85cee43d1ca81190e3b369b5a3724fe1c1...,2017-03-06 14:45:00,2017-03-06 15:00:00,1020.0,9.5,NaN,NaN,NaN,NaN,...,POINT (nan nan),POINT (nan nan),March,Monday,2017,3,0,15,15,3/5
962707,3bc4f08fcf057a6c2bd8b7865c1978abca3616ec,1721f8d69b5f831e0c16c51510786ca43aaa367d23bec8...,2017-01-16 11:45:00,2017-01-16 12:00:00,392.0,2.2,NaN,NaN,NaN,NaN,...,POINT (nan nan),POINT (nan nan),January,Monday,2017,1,0,12,12,3/5
21548285,7235674388b12e378a2bc20ad7fb565ef6a9d317,61abb47c5869d156b0658c031a16ce139beb682fc30b5b...,2017-11-04 00:00:00,2017-11-04 00:15:00,1260.0,4.7,NaN,NaN,7.0,24.0,...,POINT (41.92269 41.92269),POINT (41.90121 41.90121),November,Saturday,2017,11,5,1,121,1/5
16198848,4ab70436ec0c42ae288afb510213264fd3c6e5c3,ea73ddfdd7cafa796ab1e254ffe3240fb6936e6f390c27...,2017-08-16 12:45:00,2017-08-16 13:00:00,423.0,1.1,1.703108e+10,1.703132e+10,8.0,32.0,...,POINT (41.89503 41.89503),POINT (41.88499 41.88499),August,Wednesday,2017,8,2,13,61,3/5


## Option 1: Parrow:

In [13]:
#Write parquet of cleaned frame
df_compatible = df.drop(columns=["PU_Centroid","DO_Centroid"])
frame = pa.Table.from_pandas(df_compatible)
pq.write_table(frame, '../data/df_cleaned_{}.parquet'.format(date.today().strftime("%d_%m_%Y")))